In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
#import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [ ]:
import keras.applications
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Input
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

from keras_vggface.vggface import VGGFace

In [ ]:
import keras
#from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import Callback

model_weights_checkpoint = 'vgg_face_19_classes2.h5'

## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              verbose=0, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint(model_weights_checkpoint, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')

class Metrics(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict(self.validation_data[0]))
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        return
f1score = Metrics()

In [ ]:
from keras import backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


def precision(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    return precision


def recall(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    recall = c1 / c3

    return recall

In [ ]:
img_width, img_height = 224, 224
n_classes = 19

In [ ]:
# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(img_width, img_height, 3))  # this assumes K.image_data_format() == 'channels_last'

In [ ]:
#base_model = keras.applications.InceptionResNetV2(weights='imagenet', include_top=False)
#base_model = keras.applications.ResNet50(include_top=False, weights='imagenet')#, input_tensor=input_tensor)
#base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

base_model = VGGFace(model='resnet50', include_top=False, input_shape=(img_width, img_height, 3), pooling='avg')

In [ ]:
base_model.summary()

In [ ]:
# add a global spatial average pooling layer
x = base_model.get_layer('pool5').output

In [ ]:
# let's add a fully-connected layer
x = Flatten()(x)
x = Dense(1024, activation='tanh')(x)
x = Dropout(0.5)(x)
#x = Dense(512, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(512, activation='tanh')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(n_classes, activation='softmax')(x)

In [ ]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='sgd', loss='categorical_crossentropy')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy', precision, recall, f1_score])

In [ ]:
#model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\ir\\train'
validation_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\ir\\validation'
nb_train_samples = 31576
nb_validation_samples = 6048
epochs = 50
batch_size = 32
samples_per_epoch= 5000

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# train the model on the new data for a few epochs
#model.fit_generator(...)

# fine-tune the model
hist = model.fit_generator(
    train_generator,
    #samples_per_epoch=nb_train_samples,
    steps_per_epoch = nb_train_samples*4 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [checkpoint, history,early_stopping,f1score ])

In [ ]:
model.save("deep_transfer_19_class_vggface2.h5")

In [ ]:
from keras.preprocessing import image

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\44_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\44_6_rgb_191.png'
img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\41_10_rgb_191.png'

#img_path = 'E:\\MestradoData\\preprocessed\\all\\01\\42_1_rgb_191.png'
#img_path = 'E:\\MestradoData\\preprocessed\\all\\06\\42_6_rgb_291.png'3
#img_path = 'E:\\MestradoData\\preprocessed\\all\\10\\42_10_rgb_191.png'

img = image.load_img(img_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [ ]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

In [ ]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

In [ ]:
import datetime
a = datetime.datetime.now()
result = model.predict(x)
b = datetime.datetime.now()
print(b-a, result)

In [ ]:
test_data_dir = 'C:\\Users\\rafae\\Desktop\\Coleta\\data\\rgb\\test'
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
score = model.evaluate_generator(test_generator)
print("Accuracy = ", score)

In [ ]:
hist.history

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
predictions = model.predict_generator(
    test_generator,
    steps = np.math.ceil(test_generator.samples / test_generator.batch_size),
    verbose = 1,
    workers = 2,
    max_q_size=10,
    pickle_safe=True
    )
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
accuracy = accuracy_score(true_classes, predicted_classes)